In [ ]:
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input
from nltk.translate.bleu_score import sentence_bleu
import json
import numpy as np
import os
import pickle

print("Loading trained seq2seq model...")
model = load_model('seq2seq_model.h5')

print("Loading tokenizer...")
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

# Load max_seq_length
with open('max_seq_length.pkl', 'rb') as f:
        max_seq_length = pickle.load(f)

print("Loading preprocessed sentences...")
preprocessed_file_path = './validation/preprocessed_sentences.json'
with open(preprocessed_file_path, 'r') as f:
    preprocessed_data = json.load(f)

# Extract the encoder part of the model
print("Extracting encoder from model...")
encoder_input_layer = model.get_layer('input_1').input
encoder_output_layer, encoder_state_h, encoder_state_c = model.get_layer('lstm').output
encoder_states = [encoder_state_h, encoder_state_c]
encoder_model = Model(inputs=encoder_input_layer, outputs=encoder_states)

# Build the decoder model
print("Building decoder model...")
decoder_input = Input(shape=(None, len(tokenizer.word_index) + 1), name='decoder_input')
decoder_state_input_h = Input(shape=(128,), name='decoder_state_input_h')
decoder_state_input_c = Input(shape=(128,), name='decoder_state_input_c')
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.get_layer('lstm_1')
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(decoder_input, initial_state=decoder_states_inputs)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.get_layer('dense')
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_input] + decoder_states_inputs, [decoder_outputs] + decoder_states)


def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    print(".predict completed")
    target_seq = np.zeros((1, 1, len(tokenizer.word_index) + 1))
    target_seq[0, 0, tokenizer.word_index['<START>']] = 1.

    stop_condition = False
    decoded_sentence = ''
    iteration_count = 0

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Printing the output tokens for the current step
        print(f"[DEBUG] Iteration {iteration_count}, Output Tokens: {output_tokens}")

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = tokenizer.index_word.get(sampled_token_index)

        # Printing the selected word for the current step
        print(f"[DEBUG] Iteration {iteration_count}, Sampled Word: {sampled_char}")

        if sampled_char:
            decoded_sentence += ' ' + sampled_char

        if sampled_char == '<END>' or len(decoded_sentence.split()) > max_seq_length:
            stop_condition = True

        target_seq = np.zeros((1, 1, len(tokenizer.word_index) + 1))
        target_seq[0, 0, sampled_token_index] = 1.
        states_value = [h, c]

        iteration_count += 1

    # Printing the final decoded sentence
    print(f"[DEBUG] Decoded Sentence: {decoded_sentence}")

    return decoded_sentence


# Load video features from the directory
feature_dir = './validation/NASNetLarge_feature_vectors'
print("[INFO] Video features directory set.")

video_features = []
for item in preprocessed_data[:2]:
    file_path = os.path.join(feature_dir, f"{item['SENTENCE_NAME']}.json")
    try:
        with open(file_path, 'r') as f:
            video_features.append(json.load(f))
    except Exception as e:
        print(f"[ERROR] Failed to load: {file_path}. Error: {e}")

print("[INFO] Loaded video features.")

# Calculate BLEU scores
print("[INFO] Calculating BLEU scores...")
bleu_scores = []
original_sentences = [item['SENTENCE_DESCRIPTION'] for item in preprocessed_data[:2]]

if not video_features: 
    print("[ERROR] No video features loaded. Exiting.")
    exit()

for i, features in enumerate(video_features):
    features = np.array(features)  
    input_seq = pad_sequences([features], maxlen=max_seq_length, dtype='float32', padding='post')
    decoded_sentence = decode_sequence(input_seq)
    reference = [original_sentences[i].split()]
    candidate = decoded_sentence.split()
    score = sentence_bleu(reference, candidate)
    bleu_scores.append(score)
    print(f"[INFO] Sentence {i+1}, BLEU Score: {score}")

print(f"[INFO] Average BLEU Score: {np.mean(bleu_scores)}")
